In [1]:
import pandas as pd

import sys  
sys.path.insert(0, '/Users/cwilson/desktop/programming/jupyter_tings/healthyride/healthyride')

from resource_consolidation import Resources
from trip import Trip

from collections import Counter

from dataclasses import dataclass

from typing import List

In [2]:
url = 'https://data.wprdc.org/dataset/healthyride-trip-data'
resource_extension = '/dataset/healthyride-trip-data/resource/'
resource_calling_string = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id='


In [3]:
#create empty container for data 
data_container = Resources(url, resource_extension, resource_calling_string)
#create json string with data from API
json_list = data_container.create_json()

working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=102b0be5-aa3f-48b5-9306-e628ae1ae44d
of length: 7224
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=d46ce40f-d8d4-490d-9ab8-ba5bb91c2fbb
of length: 17595
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=030d12a3-fda6-4f2c-a508-ff23a4ecf195
of length: 26973
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=45bd5889-8ea1-4e71-ba83-f8103725e5d8
of length: 10448
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=fe985614-775c-42e3-9647-59099806329d
of length: 10
too short to add
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=c2608a60-9282-4e3d-b45d-a0306b75a3c9
of length: 11322
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=7fe58023-39be-4fa5-a7b7-27d7b0e077a7
of length: 1

In [4]:
print(json_list[0])


{'Trip id': '25723960', 'Bikeid': '70478', 'To station name': 'Zulema St & Coltart Ave', 'Usertype': 'Subscriber', 'Stoptime': '1/1/2017 8:30', 'From station name': 'S Millvale Ave & Centre Ave', 'Starttime': '1/1/2017 8:28', 'To station id': '1044', 'Tripduration': '110', '_id': 1, 'From station id': '1043'}


In [5]:
#convert list of dicts to list of objects called "Trips"
Trip_list = []
for dict in json_list:
    li = list(dict.values())
    Trip_list.append(Trip(*li))

In [6]:
#test er out
print(Trip_list[420])
print(Trip_list[420].toname)
print(Trip_list[420].starttime)
print(Trip_list[420].weekday)
print(Trip_list[420].start_neighborhood)

Trip(tripid='25783219', bikeid='70131', toname='S Euclid Ave & Centre Ave', usertype='Subscriber', stoptime='1/14/2017 12:08', fromname='42nd & Penn Ave.', starttime='1/14/2017 11:58', toid='1030', tripduration='562', _id=420, fromid='1020')
S Euclid Ave & Centre Ave
1/14/2017 11:58
5
Lawrenceville


In [7]:
def group_by_arrival_neighborhood(Trip_list):
    return_dict = {}
    for i in Trip_list:
        if i.start_neighborhood not in return_dict.keys():
            return_dict[i.start_neighborhood] = [i]
        else:
            return_dict[i.start_neighborhood].append(i)
    return return_dict


def group_by_departure_neighborhood(Trip_list):
    return_dict = {}
    for i in Trip_list:
        if i.end_neighborhood not in return_dict.keys():
            return_dict[i.end_neighborhood] = [i]
        else:
            return_dict[i.end_neighborhood].append(i)
    return return_dict

In [8]:
@dataclass
class Hour:
    self : List[Trip]
    
    def __getitem__(self, item):
         return self


    #def __post_init__(self):
        #self.counted_neighborhoods 

@dataclass
class Weekday:
    self : List[Hour]



@dataclass
class Neighborhood:
    self : List[Weekday]
    
@dataclass 
class Direction:
    arrivals : List[Neighborhood]
    departures : List[Neighborhood]

In [9]:
hour_list = Hour(Trip_list)
print(hour_list)

In [ ]:
def filter_list_to_just_objects(Trip_list):
    return [i for i in Trip_list if isinstance(i, Trip)], [i for i in Trip_list if not isinstance(i, Trip)]

#split into lists which group by day, then by hour
def group_trip_list_by_weekday(Trip_list):
    """function groups trip lists by weekday"""
    weekday_trip_dict = {}
    for i in Trip_list:
        if i.weekday not in weekday_trip_dict.keys():
            weekday_trip_dict[i.weekday] = [i]
        else:
            weekday_trip_dict[i.weekday] += [i]
    return weekday_trip_dict


def group_trip_list_by_departure_hour(Trip_list):
    """function groups trip lists by hour of day"""
    hourly_trip_dict = {}
    for i in Trip_list:
        date = i.stoptime
        try:
            hour = date[date.find(' ')+len(' '):date.rfind(':')]
        except:
            hour = 'missing date'
        
        if hour not in hourly_trip_dict.keys():
            hourly_trip_dict[hour] = [i.start_neighborhood]
        else:
            hourly_trip_dict[hour] += [i.start_neighborhood]
    return convert_hourly_trip_dict_to_counter(hourly_trip_dict)


def group_trip_list_by_arrival_hour(Trip_list):
    """function groups trip lists by hour of day"""
    #print(len(filter_list_to_just_objects(Trip_list)[1]))
    hourly_trip_dict = {}
    for i in Trip_list:
        date = i.starttime
        try:
            hour = date[date.find(' ')+len(' '):date.rfind(':')]
        except:
            hour = 'missing date'

        if hour not in hourly_trip_dict.keys():
            hourly_trip_dict[hour] = [i.end_neighborhood]
        else:
            hourly_trip_dict[hour] += [i.end_neighborhood]
    

    return convert_hourly_trip_dict_to_counter(hourly_trip_dict)



def convert_hourly_trip_dict_to_counter(hourly_trip_dict):
    counted_hourly_neighborhoods = {}
    for key, val in hourly_trip_dict.items():
        counted_hourly_neighborhoods[key] = Counter(val)
    return counted_hourly_neighborhoods
        



def create_hourly_dict_grouped_by_weeks(Trip_list, func):
    """function takes master trip list and passes it through create weekday dict, 
        then creates nested dict with hourly volumes"""
    return_hourly_values = {}
    
    weekday_trip_dict = group_trip_list_by_weekday(Trip_list)  
    
    for key, val in weekday_trip_dict.items():
        #print(key, len(filter_list_to_just_objects(val)[1]))

        return_hourly_values[key] = func(val)
    
    return return_hourly_values



In [ ]:
#apply group_by_arrival_neighborhood and group_by_departure_neighborhood first, then create hourly dict grouped by weeks
def sort_direction_neighborhood_weekday_hour(Trip_list):
    neighborhood_by_arrivals = group_by_arrival_neighborhood(Trip_list)

    neighborhood_by_departures = group_by_departure_neighborhood(Trip_list)

    return_sorted_arrivals = {}
    for key, val in neighborhood_by_arrivals.items():
        #print(key, len(filter_list_to_just_objects(val)[1]))
        return_sorted_arrivals[key] = create_hourly_dict_grouped_by_weeks(val, group_trip_list_by_arrival_hour)
    
    return_sorted_departures = {}
    for key, val in neighborhood_by_departures.items():
        #print(len(filter_list_to_just_objects(val)[1]))
        return_sorted_departures[key] = create_hourly_dict_grouped_by_weeks(val, group_trip_list_by_departure_hour)
    
    return return_sorted_arrivals, return_sorted_departures

In [ ]:
first_t = sort_direction_neighborhood_weekday_hour(Trip_list)

In [ ]:
print(first_t[0]['Oakland'][6])

In [ ]:
arrival_neighborhoods = [i.start_neighborhood for i in downtown_arrivals]
departure_neighborhoods = [i.end_neighborhood for i in downtown_departures]
print(arrival_neighborhoods[0])

In [ ]:
counted_downtown_arrivals = Counter(arrival_neighborhoods)
counted_downtown_departures = Counter(departure_neighborhoods)
print(counted_downtown_arrivals)
print(counted_downtown_departures)